### Data Loading

In [9]:
%reload_ext autoreload
%autoreload 2

In [8]:
import random
import pandas as pd
import duckdb
from tqdm import tqdm
from thefuzz import fuzz
import spacy
from spacy.matcher import Matcher
from datasets import Dataset, load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
import torch

In [3]:
papers_item = pd.read_csv('ArticlesContainingItems.csv', encoding='cp1252')
items = pd.read_csv('ISItemsandVariables.csv', encoding='cp1252')

In [4]:
db_path = r'/home/muellerrol/saras_folder/thesis/isrecon.duckdb'

In [5]:
con = duckdb.connect(r'/home/muellerrol/saras_folder/thesis/isrecon.duckdb')

In [6]:
#papers table from duckdb (the database)
papers = con.execute('SELECT * FROM papers').fetchdf()

### Merging and Preprocessing

In [7]:
# Matching papers and papers_item on title and articletitle
papers_item_merged = papers_item.merge(papers, left_on='articletitle', right_on='title')


In [8]:
# Processing papers to get items relevant to each article
papers = con.execute('SELECT * FROM papers INNER JOIN authors USING (article_id) WHERE author_position = 1').fetchdf()
is_papers_item = papers_item[papers_item['journalname'].isin(papers['journal'].unique())]
is_papers_item['title_short'] = is_papers_item['articletitle'].str.lower().str.replace('[^a-z0-9]', '').str[:20]
papers['title_short'] = papers['title'].str.lower().str.replace('[^a-z0-9]', '').str[:20]
is_papers_item.rename(columns={'articletitle': 'title', 'journalname': 'journal'}, inplace=True)
is_merged = is_papers_item.merge(papers, left_on=['year', 'journal', 'title_short'], right_on=['year', 'journal', 'title_short'])

/tmp/ipykernel_106663/3294962847.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_papers_item['title_short'] = is_papers_item['articletitle'].str.lower().str.replace('[^a-z0-9]', '').str[:20]
/tmp/ipykernel_106663/3294962847.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_papers_item.rename(columns={'articletitle': 'title', 'journalname': 'journal'}, inplace=True)


### Data Extraction and Preparation

In [9]:
item_sources_columns = ['article_id', 'SourceId', 'ItemId', 'Definition']
item_sources = []

for i, row in tqdm(is_merged.iterrows(), total=len(is_merged)):
    source_id = row['SourceId']
    article_id = row['article_id']
    items_in_source = items[items['SourceId'] == source_id]
    for _, item in items_in_source.iterrows():
        row = [article_id, source_id, item['ItemId'], item['Definition']] 
        item_sources.append(row)

item_sources_df = pd.DataFrame(item_sources, columns=item_sources_columns)

 25%|██▌       | 216/858 [00:00<00:00, 2156.37it/s]

100%|██████████| 858/858 [00:01<00:00, 685.48it/s] 


In [10]:
# Loading sentences table from the database
sentences = con.execute('SELECT * FROM sentences').fetchdf()
sentences = sentences[sentences['article_id'].isin(item_sources_df['article_id'])]


### Fuzzy Matching

In [11]:
# Defining the fuzzy match function
def fuzzy_match(sentence, definition_list, threshold=85):
    found = False
    best_definition = None
    best_ratio = 0
    for definition in definition_list:
        if len(sentence) + 2 >= len(definition):
            ratio = fuzz.partial_ratio(sentence, definition)
            if ratio >= threshold and ratio > best_ratio:
                best_ratio = ratio
                best_definition = definition
                found = True
    return found, best_definition, best_ratio


In [12]:
all_matched_definitions = []

In [13]:
definition_list = list(item_sources_df['Definition'])

In [14]:
# Get unique article IDs
unique_article_ids = list(item_sources_df['article_id'].unique())

# unique article IDs iteration
for article_id in tqdm(unique_article_ids, total=len(unique_article_ids)):
   
    article_sentences = sentences[sentences['article_id'] == article_id]

    definitions_in_article = item_sources_df[item_sources_df['article_id'] == article_id]
    
    # Filter out any NaN or non-strings
    definition_list = list(definitions_in_article['Definition'].dropna().astype(str))

    # Fuzzy matching
    for index, article_sentence in article_sentences.iterrows():
        sentence_original = article_sentence['sentence_original']
        if isinstance(sentence_original, str): 
            matched, definition, ratio = fuzzy_match(sentence_original, definition_list)
            
            if matched:
                row = article_sentence.copy()
                row['matched_definition'] = definition
                row['ratio'] = ratio
                row['definition_id'] = definitions_in_article[definitions_in_article['Definition'] == definition].iloc[0]['ItemId']  # Get ItemId or other identifier
                row['definition_sentence'] = True
                all_matched_definitions.append(row.to_frame().T)
            else:
                
                if random.random() < 0.1:
                    row = article_sentence.copy()
                    row['matched_definition'] = None
                    row['ratio'] = None
                    row['definition_id'] = None
                    row['definition_sentence'] = False
                    all_matched_definitions.append(row.to_frame().T)
all_matched_definitions_df = pd.concat(all_matched_definitions, ignore_index=True)


  0%|          | 1/446 [00:00<03:12,  2.32it/s]

100%|██████████| 446/446 [03:01<00:00,  2.45it/s]


### Data Saving

In [15]:
all_matched_definitions_df.to_csv('/home/muellerrol/saras_folder/thesis/extracted_definitions.csv', index=False)

### Checking and Removing Duplicates

In [4]:
# Remove duplicated rows
data = data.drop_duplicates().reset_index(drop=True)
print("Duplicates removed. Updated dataset size:", len(data))

# Duplicates check
duplicates = data.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")


Duplicates removed. Updated dataset size: 31244
Number of duplicate rows: 0
